In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predictions/predictions.json
/kaggle/input/tensorflow2-question-answering/sample_submission.csv
/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl
/kaggle/input/tensorflow2-question-answering/simplified-nq-train.jsonl


In [85]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import sys
import collections
import json

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/predictions/predictions.json
/kaggle/input/tensorflow2-question-answering/sample_submission.csv
/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl
/kaggle/input/tensorflow2-question-answering/simplified-nq-train.jsonl


In [86]:
answers_df = pd.read_json("../input/predictions/predictions.json")
answers_df.head()
answers_df["predictions"][1]

{'example_id': '8777415633185303067',
 'long_answer': {'start_byte': -1,
  'end_byte': -1,
  'start_token': 1462,
  'end_token': 1760},
 'long_answer_score': 0.708450376987457,
 'short_answers': [{'start_byte': -1,
   'end_byte': -1,
   'start_token': 1634,
   'end_token': 1644}],
 'short_answers_score': 0.009059190750122001,
 'yes_no_answer': 'NONE'}

In [87]:
# def df_long_index_score(df):
#     answers = []
#     cont = 0
#     for e in df['long_answer_score']['long_answer']:
#         if e > 0: 
#             index = {}
#             index['start'] = e[0]
#             index['end'] = e[1]
#             index['score'] = e[2]
#             answers.append(index)
#             cont += 1
#         # number of answers
#         if cont == 1:
#             break
            
#     return answers

# def df_short_index_score(df):
#     answers = []
#     cont = 0
#     for e in df['short_answers_score']:
#         # if score > 2
#         if e[2] > 8:
#             index = {}
#             index['start'] = e[0]
#             index['end'] = e[1]
#             index['score'] = e[2]
#             answers.append(index)
#             cont += 1
#         # number of answers
#         if cont == 1:
#             break
            
#     return answers

# def df_example_id(df):
#     return df['example_id']

In [88]:
def df_long_index_score(df):
    answers = []
    cont = 0
    for i in range(len(df)):
        index={}
        if df["predictions"][i]['long_answer_score']>0:
            index['start'] = df["predictions"][i]['long_answer']['start_token']
            index['end'] = df["predictions"][i]['long_answer']['end_token']
            index['score'] = df["predictions"][i]['long_answer_score']
            index = [index]
            answers.append(index) 
        else:
            answers.append([])    
    return answers

def df_short_index_score(df):
    answers = []
    cont = 0
    for i in range(len(df)):
        index={}
        if df["predictions"][i]['short_answers_score']>0:
            index['start'] = df["predictions"][i]['short_answers'][0]['start_token']
            index['end'] = df["predictions"][i]['short_answers'][0]['end_token']
            index['score'] = df["predictions"][i]['short_answers_score']
            index = [index]
            answers.append(index) 
        else:
            answers.append([]) 
    return answers

In [89]:
def df_example_id(df):
    return df['example_id']

In [90]:
# answers_df['answer'] = answers_df['predictions'].apply(df_long_index_score)
answers_df['long_indexes_and_scores']=df_long_index_score(answers_df)
answers_df['short_indexes_and_scores']=df_short_index_score(answers_df)


In [91]:
answers_df['example_id'] = answers_df['predictions'].apply(df_example_id)


In [92]:
answers_df

,predictions,long_indexes_and_scores,short_indexes_and_scores,example_id
0,"{'example_id': '-1220107454853145579', 'long_a...",[],[],-1220107454853145579
1,"{'example_id': '8777415633185303067', 'long_an...","[{'start': 1462, 'end': 1760, 'score': 0.70845...","[{'start': 1634, 'end': 1644, 'score': 0.00905...",8777415633185303067
2,"{'example_id': '4640548859154538040', 'long_an...","[{'start': 165, 'end': 272, 'score': 0.7363434...","[{'start': 204, 'end': 220, 'score': 0.2330710...",4640548859154538040
3,"{'example_id': '-5316095317154496261', 'long_a...",[],[],-5316095317154496261
4,"{'example_id': '-8752372642178983917', 'long_a...","[{'start': 1380, 'end': 1790, 'score': 0.91983...","[{'start': 1382, 'end': 1384, 'score': 0.03239...",-8752372642178983917
...,...,...,...,...
341,"{'example_id': '934950704129184964', 'long_ans...","[{'start': 5006, 'end': 5197, 'score': 0.91472...","[{'start': 5019, 'end': 5020, 'score': 0.89214...",934950704129184964
342,"{'example_id': '-7473086642300315642', 'long_a...","[{'start': 318, 'end': 428, 'score': 0.5696351...","[{'start': 332, 'end': 336, 'score': 0.2683105...",-7473086642300315642
343,"{'example_id': '4153219118929483698', 'long_an...","[{'start': 1641, 'end': 1680, 'score': 0.56105...","[{'start': -1, 'end': -1, 'score': 0.530325997...",4153219118929483698
344,"{'example_id': '-7341065354441938421', 'long_a...","[{'start': 116, 'end': 792, 'score': 0.6383808...","[{'start': 155, 'end': 159, 'score': 0.2582953...",-7341065354441938421


In [93]:
answers_df = answers_df.drop(['predictions'], axis=1)

In [94]:
answers_df.head()

,long_indexes_and_scores,short_indexes_and_scores,example_id
0,[],[],-1220107454853145579
1,"[{'start': 1462, 'end': 1760, 'score': 0.70845...","[{'start': 1634, 'end': 1644, 'score': 0.00905...",8777415633185303067
2,"[{'start': 165, 'end': 272, 'score': 0.7363434...","[{'start': 204, 'end': 220, 'score': 0.2330710...",4640548859154538040
3,[],[],-5316095317154496261
4,"[{'start': 1380, 'end': 1790, 'score': 0.91983...","[{'start': 1382, 'end': 1384, 'score': 0.03239...",-8752372642178983917


In [95]:
def create_answer(entry):
    answer = []
    for e in entry:
        answer.append(str(e['start']) + ':'+ str(e['end']))
    if not answer:
        answer = ""
    return ", ".join(answer)

In [96]:
answers_df["long_answer"] = answers_df['long_indexes_and_scores'].apply(create_answer)
answers_df["short_answer"] = answers_df['short_indexes_and_scores'].apply(create_answer)
answers_df["example_id"] = answers_df['example_id'].apply(lambda q: str(q))

long_answers = dict(zip(answers_df["example_id"], answers_df["long_answer"]))
short_answers = dict(zip(answers_df["example_id"], answers_df["short_answer"]))

answers_df.head()

,long_indexes_and_scores,short_indexes_and_scores,example_id,long_answer,short_answer
0,[],[],-1220107454853145579,,
1,"[{'start': 1462, 'end': 1760, 'score': 0.70845...","[{'start': 1634, 'end': 1644, 'score': 0.00905...",8777415633185303067,1462:1760,1634:1644
2,"[{'start': 165, 'end': 272, 'score': 0.7363434...","[{'start': 204, 'end': 220, 'score': 0.2330710...",4640548859154538040,165:272,204:220
3,[],[],-5316095317154496261,,
4,"[{'start': 1380, 'end': 1790, 'score': 0.91983...","[{'start': 1382, 'end': 1384, 'score': 0.03239...",-8752372642178983917,1380:1790,1382:1384


In [97]:
answers_df = answers_df.drop(['long_indexes_and_scores', 'short_indexes_and_scores'], axis=1)
answers_df.head()


,example_id,long_answer,short_answer
0,-1220107454853145579,,
1,8777415633185303067,1462:1760,1634:1644
2,4640548859154538040,165:272,204:220
3,-5316095317154496261,,
4,-8752372642178983917,1380:1790,1382:1384


In [98]:
sample_submission = pd.read_csv("../input/tensorflow2-question-answering/sample_submission.csv")

long_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_long")].apply(lambda q: long_answers[q["example_id"].replace("_long", "")], axis=1)
short_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_short")].apply(lambda q: short_answers[q["example_id"].replace("_short", "")], axis=1)

sample_submission.loc[sample_submission["example_id"].str.contains("_long"), "PredictionString"] = long_prediction_strings
sample_submission.loc[sample_submission["example_id"].str.contains("_short"), "PredictionString"] = short_prediction_strings

In [99]:
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission